In [44]:
import keras
from keras.layers import Input ,Dense, Dropout, Activation, LSTM
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import Adam
from keras.engine.network import Network

from keras.initializers import glorot_normal,orthogonal

from keras.callbacks import EarlyStopping
import csv

from PIL import Image

import numpy as np
import time

In [36]:
#どんくらいで学習させるか関連x_test
num_sample=100 #画像の総フレーム数
timesteps=30 #一回のLSTMに入れる値の数
camera=5 #カメラの数

#画像関連
channels=3
img_width=100
img_height=100

# LSTMなどで用いる値
n_hidden = 256    # 出力次元
epochs = 100      # エポック数
batch_size = 5   # ミニバッチサイズ

In [38]:
class Data:
    def __init__(self, camera_num, file_name, samples, timesteps, width, height):
        self.camera_num = camera_num
        self.file_name =file_name
        self.samples=samples
        self.timesteps=timesteps
        self.width=width
        self.height=height
        self.learnData=[]
        self.labelData=[]
        
        self.make_selection()
        self.make_image()
    
    def make_selection(self):
        csv_selection=csv.reader(open(self.file_name+"video/mintime_optimize.csv", 'r'))
        one_hot = np.eye(self.camera_num)
        for row2 in csv_selection:
            self.labelData.append(one_hot[int(row2[1])])
    
    def make_image(self):
        for i in range(0, self.camera_num):
            img_list=[]
            print()
            for j in range(0,self.samples):
                #どのくらいデータロードが進んでいるか
                pro_size=20
                bar = int(j*pro_size/self.samples)
                pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
                percent ='{:03f}'.format(j / self.samples * 100.)
                print('\r{0}/{1} [{2}] {3}%'.format((i+1), self.camera_num, pro_bar, percent), end='')
                time.sleep(0.5)
                
                file_path = "image/"+str(i)+"/"+str(j)+".jpg"
                img = Image.open(file_path).convert('RGB') ## Gray->L, RGB->RGB
                img = img.resize((self.width, self.height))
                x = np.array(img, dtype=np.float32)
                x = x / 255.
                
                img_list.append(x)
            self.learnData.append(img_list)
    
    def get_learndata(self, num):
        test=[]
        for i in range(0,self.samples-self.timesteps):
            tmp_list=[]
            for j in range(0, self.timesteps):
                tmp_list.append(self.learnData[num][i+j])
            test.append(tmp_list)
        tmp=np.array(test)
        tmp=tmp.astype(np.float)
                
        return tmp
    
    def get_labeldata(self):
        test=[]
        for i in range(self.timesteps,self.samples):
            test.append(self.labelData[i])
        tmp = np.array(test)
        tmp = tmp.astype(np.float)
        
        return tmp
    
    def show(self):
        print()
        #print(self.get_learndata(0).shape)
        print(self.get_labeldata().shape)

In [39]:
data = Data(camera, "./", num_sample, timesteps, img_width, img_height)
data.show()


5/5 [=================== ] 99.000000%
(70, 5)


In [58]:
class Prediction :
    def __init__(self, timesteps, n_hidden, n_out, width, height):
        self.maxlen = timesteps
        self.n_hidden = n_hidden
        self.n_out = n_out
        self.width=width
        self.height=height
        self.sharedLayer = self.create_sharedmodel()
    
    
    def _conv_bn_relu(nb_filter, nb_row, nb_col, subsample=(1, 1)):
        def f(input):
            conv = Convolution2D(nb_filter=nb_filter, nb_row=nb_row, nb_col=nb_col, subsample=subsample,
                                 init="he_normal", border_mode="same")(input)
            norm = BatchNormalization(mode=0, axis=1)(conv)
            return Activation("relu")(norm)
        return f
    
    def _bn_relu_conv(nb_filter, nb_row, nb_col, subsample=(1, 1)):
        def f(input):
            norm = BatchNormalization(mode=0, axis=1)(input)
            activation = Activation("relu")(norm)
            return Convolution2D(nb_filter=nb_filter, nb_row=nb_row, nb_col=nb_col, subsample=subsample,
                                 init="he_normal", border_mode="same")(activation)
        return f

    def _basic_block(nb_filters, init_subsample=(1, 1)):
        def f(input):
            conv1 = _bn_relu_conv(nb_filters, 3, 3, subsample=init_subsample)(input)
            residual = _bn_relu_conv(nb_filters, 3, 3)(conv1)
            return _shortcut(input, residual)
        return f

    def _shortcut(input, residual):
        stride_width = input._keras_shape[2] / residual._keras_shape[2]
        stride_height = input._keras_shape[3] / residual._keras_shape[3]
        equal_channels = residual._keras_shape[1] == input._keras_shape[1]

        shortcut = input
        if stride_width > 1 or stride_height > 1 or not equal_channels:
            shortcut = Convolution2D(nb_filter=residual._keras_shape[1], nb_row=1, nb_col=1,
                                     subsample=(stride_width, stride_height),init="he_normal", border_mode="valid")(input)
        return merge([shortcut, residual], mode="sum")

    def _residual_block(block_function, nb_filters, repetations, is_first_layer=False):
        def f(input):
            for i in range(repetations):
                init_subsample = (1, 1)
                if i == 0 and not is_first_layer:
                    init_subsample = (2, 2)
                    input = block_function(nb_filters=nb_filters, init_subsample=init_subsample)(input)
            return input
        return f
    
    def create_sharedmodel(self):
        inputs = Input(shape=(self.maxlen, self.height, self.width, 3))
        
        x = TimeDistributed(self._conv_bn_relu(nb_filter=64, nb_row=7, nb_col=7, subsample=(2, 2)))(inputs)
        x = TimeDistributed(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), border_mode="same"))(x)
        block_fn = self._basic_block
        x = TimeDistributed(self._residual_block(block_fn, nb_filters=64, repetations=3, is_first_layer=True))(x)
        x = TimeDistributed(self._residual_block(block_fn, nb_filters=128, repetations=2))(x)
        x = TimeDistributed(self._residual_block(block_fn, nb_filters=256, repetations=2))(x)
        x = TimeDistributed(self._residual_block(block_fn, nb_filters=512, repetations=2))(x)
        x = TimeDistributed(GlobalAveragePooling2D(pool_size=(7, 7), strides=(1, 1), border_mode="same"), name="GAP")(x)
        x = TimeDistributed(Flatten())(x)
        x = TimeDistributed(Dense(512), name="dense")(x)
        predictions = LSTM(self.n_hidden, batch_input_shape = (None, self.maxlen, 35),
             kernel_initializer = glorot_normal(seed=20181020),
             recurrent_initializer = orthogonal(gain=1.0, seed=20181020), 
             dropout = 0.01, 
             recurrent_dropout = 0.01)(x)
        #predictions = Dense(256, activation = "softmax", name="time_distr_dense_one")(x)
        
        shared_layers = Network(inputs, predictions, name="shared_layers")
        return shared_layers
    
    def create_model(self):
        model_input1 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input2 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input3 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input4 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input5 = Input(shape=(self.maxlen, self.height, self.width, 3))
        
        mid_feature1 = self.sharedLayer(model_input1)
        mid_feature2 = self.sharedLayer(model_input2)
        mid_feature3 = self.sharedLayer(model_input3)
        mid_feature4 = self.sharedLayer(model_input4)
        mid_feature5 = self.sharedLayer(model_input5)
        
        merged_vector = keras.layers.concatenate([mid_feature1, mid_feature2, mid_feature3, mid_feature4, mid_feature5], axis=-1)
        
        mid_dense = Dense(self.n_hidden, activation='sigmoid')(merged_vector)
        predictions = Dense(self.n_out, activation='sigmoid')(mid_dense)
        
        model = Model(inputs=[model_input1, model_input2, model_input3, model_input4, model_input5], outputs=predictions)
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
        
        return model
    
    def train(self, x_train1, x_train2, x_train3, x_train4, x_train5, t_train, batch_size, epochs) :
        early_stopping = EarlyStopping(patience=5, verbose=1)
        model = self.create_model()
        hist = model.fit([x_train1, x_train2, x_train3, x_train4, x_train5], t_train, batch_size = batch_size, epochs = epochs, verbose = 1,
              shuffle = True, callbacks = [early_stopping], validation_split = 0.3)
        return model, hist

In [40]:
# モデル定義
pred = Prediction(timesteps, n_hidden, camera, img_width, img_height)
# 学習
model , hist = pred.train(data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4), 
                                data.get_labeldata(), batch_size, epochs)
# テスト
score = model.evaluate([data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4)], 
                                data.get_labeldata(), batch_size = batch_size, verbose = 1)
print("score:", score)

pre = [[0 for i in range(3)] for j in range(3)]
# 正答率集計
preds = model.predict([data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4)])
correct = 0
for i in range(len(preds)):
    pred = np.argmax(preds[i,:])
    tar = np.argmax(data.get_labeldata()[i,:])
    pre[pred][tar]+=1
    if pred == tar :
        correct += 1

print("正答率:", 1.0 * correct / len(preds))

Train on 49 samples, validate on 21 samples
Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[716800,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense_4/random_uniform/RandomUniform (defined at /home/kei666/.pyenv/versions/3.6.4/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4139)  = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=5374849, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_4/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_4/random_uniform/RandomUniform', defined at:
  File "/home/kei666/.pyenv/versions/3.6.4/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kei666/.pyenv/versions/3.6.4/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kei666/.pyenv/versions/3.6.4/lib/py

In [59]:
pred = Prediction(timesteps, n_hidden, camera, img_width, img_height)
model = pred.create_model()
print(model.summary())

TypeError: _conv_bn_relu() got multiple values for argument 'nb_filter'